In [1]:
# Imports and Dependencies
import pandas as pd
import math as m
pd.set_option('display.max_rows', None)
nfa = pd.read_csv("NFA_Data.csv")
# Reads in csv for table two of the UN data, using the fifth row as header.
un = pd.read_csv("UN_Data_Table_2.csv", header=4).drop([0,67,121,159,193])

C:\Users\mrjac\Anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# NFA

In [2]:
# Creating country, year, and carbon emissions dataframe for the selected years.
nfa = nfa[['country','year','carbon','record']]
nfa_data1990 = nfa[nfa['year'] == 1990]
nfa_data2000 = nfa[nfa['year'] == 2000]
nfa_data2010 = nfa[nfa['year'] == 2010]

# Creating combined dataset.
nfa = ((nfa_data1990.append(nfa_data2000)).append(nfa_data2010))

# Dropping from country names.
nfa = nfa[nfa['country'] != 'World']

# Reseting index values.
nfa = nfa.set_index("country").reset_index()

In [3]:
nfa = nfa[(nfa['record'] == 'EFProdPerCap') | (nfa['record'] == 'EFConsPerCap')]
nfa.dropna(how='any',inplace=True)
nfa = nfa.set_index("country").reset_index()

In [4]:
# Reformats NFA data.
i = 0
country_col = []
year_col = []
c_cons_col = []
c_prods_col = []

for row in nfa['country']:
    if i < len(nfa['country']):
        country_col.append(nfa['country'][i])
        year_col.append(nfa['year'][i])
        c_cons_col.append(nfa['carbon'][i])
        i += 1
        c_prods_col.append(nfa['carbon'][i])
        i += 1
    
nfa = pd.DataFrame()
nfa['country'] = country_col
nfa['year'] = year_col
nfa['c_consumed'] = c_cons_col
nfa['c_produced'] = c_prods_col

# UN

In [5]:
# Selects only columns with valid, single year names.
selected_columns = []
for column in un:
    if 'Unnamed' not in column:
        if '-' not in column:
            selected_columns.append(column)

# Turns .. into NaN values. Drops rows with only NaN values.
un = un.replace("..","NaN") 
un = (un[selected_columns])
un.dropna(thresh = 9, inplace = True)

# Drops bottom rows of summary data.
un.drop(un.tail(36).index,inplace=True)

# Transforms data from horizontal to vertical.
un = pd.melt(un, id_vars = 'Country', value_vars = ['1990','2000','2010', '2014', '2015'])

# Renames columns.
un = un.rename(columns = {'Country':'country','variable':'year','value':'hdi'})

# Final Dataset

In [6]:
un['country'] = un['country'].astype('string')
nfa['country'] = nfa['country'].astype('string')
un['year'] = un['year'].astype('int64')
nfa['year'] = nfa['year'].astype('int64')

In [7]:
data = nfa.merge(un, on = ['country','year'])

In [10]:
data.sort_values('country').set_index('country').reset_index()
barbados = data[data['country'] == 'Barbados']

In [11]:
barbados

,country,year,c_consumed,c_produced,hdi
8,Barbados,1990,2.428270,1.403260,0.732
95,Barbados,2000,2.974244,1.499070,0.771
197,Barbados,2010,3.435050,1.889276,0.797
